In [ ]:
import pandas as pd
import openai
import json

client = openai.OpenAI(api_key='YOUR API KEY') 

file_path = 'ABA Dataset - Label Topic (For Tasks 1 & 2).xlsx'
df = pd.read_excel(file_path)

df = df.rename(columns={'Unnamed: 0': 'ID'})

df_positive = df[['ID', 'PositiveReview']].rename(columns={'PositiveReview': 'Review'})
df_negative = df[['ID', 'NegativeReview']].rename(columns={'NegativeReview': 'Review'})

# Remove empty reviews
df_positive = df_positive[df_positive['Review'].notna() & (df_positive['Review'].str.strip() != "")]
df_negative = df_negative[df_negative['Review'].notna() & (df_negative['Review'].str.strip() != "")]

df_positive = df_positive.drop_duplicates()
df_negative = df_negative.drop_duplicates()

df_combined = pd.concat([df_positive, df_negative], ignore_index=True)
df_combined = df_combined.sort_values(by=['ID']).reset_index(drop=True)


df_combined.head(30)

In [ ]:
def parse_response(response_json, review_id):
    results = []
    try:
        parsed = json.loads(response_json)
        topics_list = parsed["Topics"]
        for topic_group in topics_list:
            for topic, entries in topic_group.items():
                for entry in entries:
                    text = entry.get("text") or ""
                    label = entry.get("label") or ""
                    
                    results.append({
                        "ID": review_id,
                        "Topics": topic,
                        "Text": text,
                        "NegPos": label
                    })
    except Exception as e:
        print(f"Failed to parse response for ID {review_id}: {e}")
    return results

In [ ]:
# 7-shot

def send_prompt(review_text):
    messages = [
        {
            "role": "system",
            "content": (
                'Please output the following [text] according to the [constraints] in the [output format].\n '
                '[constraints]* The output should only be in the [output format], and you must classify which part of the text corresponds to which Topic in the [Topics]. '
                'Additionally, determine whether each classified element is Positive or Negative. If there is no corresponding element, put Null for both `text` and `label`. '
                'The most important constraint is not to include any extra characters such as newline characters, `json`, or backticks, or any other unnecessary text outside of the [output format]. '
                'If there are two or more elements of the same Topic, number each so that they do not conflict when converted to json format data. '
                'However, if they have the same NegPos label, keep them in one Text as much as possible.* \n '
                '[Topics] Room, Staff, Location, Food, Price, Facility, Check-in, Check-out \n\n '
                '[output format] '
                '{"Topics":[{"Room":[{"text": "test","label": "Positive"}],'
                '"Staff":[{"text": null,"label": null}],'
                '"Location":[{"text": "test","label": "Positive"}],'
                '"Food":[{"text": "test","label": "Negative"}],'
                '"Price":[{"text": "test","label": "Positive"}],'
                '"Facility":[{"text": "test","label": "Negative"}],'
                '"Check-in":[{"text": "test","label": "Positive"}],'
                '"Check-out":[{"text": null,"label": null}]}]}'
            )
        },
        {
            "role": "user",
            "content": "The room is enough big. But the room was a little bit durty."
        },
        {
            "role": "assistant",
            "content": '{"Topics":[{"Room1":[{"text": "The room is enough big.","label": "Positive"}],'
                       '"Room2":[{"text": "the room was a little bit durty.","label": "Negative"}],'
                       '"Staff":[{"text": null,"label": null}],'
                       '"Location":[{"text": null,"label": null}],'
                       '"Food":[{"text": null,"label": null}],'
                       '"Price":[{"text": null,"label": null}],'
                       '"Facility":[{"text": null,"label": null}],'
                       '"Check-in":[{"text": null,"label": null}],'
                       '"Check-out":[{"text": null,"label": null}]}]}'
        },
        {
            "role": "user",
            "content": "The room was very clean cheap, well decorated and modern, although not big."
        },
        {
            "role": "assistant",
            "content": '{"Topics":[{"Room1":[{"text": "The room was very clean, well decorated and modern","label": "Positive"}],'
                       '"Room2":[{"text": "although not big","label": "Negative"}],'
                       '"Price":[{"text": "cheap","label": "Positive"}],'
                       '"Staff":[{"text": null,"label": null}],'
                       '"Location":[{"text": null,"label": null}],'
                       '"Food":[{"text": null,"label": null}],'
                       '"Facility":[{"text": null,"label": null}],'
                       '"Check-in":[{"text": null,"label": null}],'
                       '"Check-out":[{"text": null,"label": null}]}]}'
        },
        {
          "role": "user",
          "content": "The room was spacious, bright, and tastefully decorated but smelled musty. The bed was large and comfortable, but the sheets were not very clean. Air conditioning worked great, but was a bit noisy. Location is central, but very crowded. Price was reasonable for the quality. Breakfast was delicious but expensive."
        },
        {
          "role": "assistant",
          "content": '{"Topics":[{"Room1":[{"text": "The room was spacious, bright, and tastefully decorated.","label": "Positive"}],'
                               '"Room2":[{"text": "The room smelled musty.","label": "Negative"}],'
                               '"Room3":[{"text": "The bed was large and comfortable.","label": "Positive"}],'
                               '"Room4":[{"text": "The sheets were not very clean.","label": "Negative"}],'
                               '"Room5":[{"text": "The air conditioning worked great.","label": "Positive"}],'
                               '"Room6":[{"text": "The air conditioning was a bit noisy.","label": "Negative"}],'
                               '"Staff":[{"text": null,"label": null}],'
                               '"Location1":[{"text": "Location is central.","label": "Positive"}],'
                               '"Location2":[{"text": "Location was very crowded.","label": "Negative"}],'
                               '"Food1":[{"text": "Breakfast was delicious.","label": "Positive"}],'
                               '"Food2":[{"text": "Breakfast was expensive.","label": "Negative"}],'
                               '"Price":[{"text": "Price was reasonable for the quality.","label": "Positive"}],'
                               '"Facility":[{"text": null,"label": null}],'
                               '"Check-in":[{"text": null,"label": null}],'
                               '"Check-out":[{"text": null,"label": null}]}]}'
        },
        {
          "role": "user",
          "content": "Wonderful experience! Would definitely stay here again. Perfect place for a honeymoon. I traveled for work and found it convenient."
        },
        {
          "role": "assistant",
          "content": '{"Topics":[{"Room":[{"text": null,"label": null}],'
                               '"Staff":[{"text": null,"label": null}],'
                               '"Location":[{"text": null,"label": null}],'
                               '"Food":[{"text": null,"label": null}],'
                               '"Price":[{"text": null,"label": null}],'
                               '"Facility":[{"text": "Perfect place for a honeymoon.","label": null}],'
                               '"Check-in":[{"text": null,"label": null}],'
                               '"Check-out":[{"text": null,"label": null}]}]}'
        },
        {
          "role": "user",
          "content": "Location. The hotel was close to the airport, which made traveling easy. However, there was a lot of street noise outside the window. Staff. The receptionist was polite and friendly. However, check-in took longer than expected. The hotel lobby was welcoming and spacious. The room had a comfortable bed, but the air conditioning was loud at night. The neighbors were noisy through the walls, and the WiFi in the room was weak and unreliable. The breakfast buffet was delicious; however, the coffee was terrible. The price was reasonable for the quality. The building was charming with historical architecture."
        },
        {
          "role": "assistant",
          "content": '{"Topics":[{"Room1":[{"text": "The room had a comfortable bed.","label": "Positive"}],'
                               '"Room2":[{"text": "The air conditioning was loud at night.","label": "Negative"}],'
                               '"Room3":[{"text": "The neighbors were noisy through the walls.","label": "Negative"}],'
                               '"Room4":[{"text": "The WiFi in the room was weak and unreliable.","label": "Negative"}],'
                               '"Staff":[{"text": "The receptionist was polite and friendly.","label": "Positive"}],'
                               '"Location1":[{"text": "The hotel was close to the airport, which made traveling easy.","label": "Positive"}],'
                               '"Location2":[{"text": "There was a lot of street noise outside the window.","label": "Negative"}],'
                               '"Food1":[{"text": "The breakfast buffet was delicious.","label": "Positive"}],'
                               '"Food2":[{"text": "The coffee was terrible.","label": "Negative"}],'
                               '"Price":[{"text": "The price was reasonable for the quality.","label": "Positive"}],'
                               '"Facility1":[{"text": "The hotel lobby was welcoming and spacious.","label": "Positive"}],'
                               '"Facility2":[{"text": "The building was charming with historical architecture.","label": "Positive"}],'
                               '"Check-in":[{"text": "check-in took longer than expected.","label": "Negative"}],'
                               '"Check-out":[{"text": null,"label": null}]}]}'
        },
         {
          "role": "user",
          "content": "location, service, overall was good, Sure worth it to come back again"
        },
        {
          "role": "assistant",
          "content": '{"Topics":[{"Room":[{"text": null,"label": null}],'
                                '"Staff":[{"text": null,"label": null}],'
                                '"Location":[{"text": null,"label": null}],'
                                '"Food":[{"text": null,"label": null}],'
                                '"Price":[{"text": null,"label": null}],'
                                '"Facility":[{"text": null,"label": null}],'
                                '"Check-in":[{"text": null,"label": null}],'
                                '"Check-out":[{"text": null,"label": null}]}]}'

        },
        {
          "role": "user",
          "content": "The apartment was new. The breakfast was amazing and the price was quite reasonable.Overall, we definitely planning to return again soon!"
        },
        {
          "role": "assistant",
          "content": '{"Topics":[{"Room1":[{"text": null,"label": null}],'
                     '"Room":[{"text": null,"label": null}],'
                     '"Staff":[{"text": null,"label": null}],'
                     '"Location":[{"text": null,"label": null}],'
                     '"Food":[{"text": "The breakfast was amazing and the price was quite reasonable.","label": "Positive"}],'
                     '"Price":[{"text": null,"label": null}],'
                     '"Facility":[{"text": "The apartment was new.","label": "Positive"}],'
                     '"Check-in":[{"text": null,"label": null}],'
                     '"Check-out":[{"text": null,"label": null}]}]}'
        },
        {
            "role": "user",
            "content": review_text
        }
        
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o", 
        messages=messages,
        temperature=0,
        top_p=0.05
    )
    return response.choices[0].message.content

final_results = []
for idx, row in df.iterrows():
    review_text = row['Review']
    review_id = row['ID']
    print(f"Processing review ID {review_id}...")

    try:
        response_content = send_prompt(review_text)
        print(f"Raw response for ID {review_id}:\n{response_content}\n")
        
        review_results = parse_response(response_content, review_id)
        final_results.extend(review_results)
    except Exception as e:
        print(f"Error processing ID {review_id}: {e}")


output_df = pd.DataFrame(final_results)
output_csv_path = 'output_formatted_reviews_gpt4o_7_v1_3.csv'
output_df.to_csv(output_csv_path, index=False)

print(f"Done! Responses saved to {output_csv_path}")
